In [15]:
"""
Code to draw vector arrows in the 3D plot
"""
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.patches import FancyArrowPatch

class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs)

def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

setattr(Axes3D, 'arrow3D', _arrow3D)

In [16]:
import numpy as np
import sim_constants as const
from numba import njit, prange
import scipy.integrate as INT
import time

# UNKNOWN IMPORT - meant for files?
# from tkinter.filedialog import askopenfilename
# import tkinter

@njit
def rand_v(mu, sigma):
    """
    A random speed generator based on light gas particles at a desired temperature with mean and std 
    given by the maxwell boltzman distribution.
    
    Params
    ------
    mu : float
        the mean of the maxwell-boltzman distribution
    sigma : float
        the standard deviation of the maxwell-boltzman distribution
    
    Return: float
        a scalar value for the speed drawn from a normal distribution
    """
    # The distribution in terms of the vectorial velocity is a normal distribution
    return np.random.normal(mu, sigma)
    
@njit
def rand_3D_vect():
    """
    Generates a random 3D unit vector (direction) with a uniform spherical distribution.
    Algo from http://stackoverflow.com/questions/5408276/python-uniform-spherical-distribution.
    
    Return
    ------
    vect : np.array
        a 3x1 vector array with magnitude 1 
    """
    vect = np.zeros(3)
    phi = np.random.uniform(0, np.pi*2)
    costheta = np.random.uniform(-1, 1)

    theta = np.arccos(costheta)
    
    vect[0] = np.sin(theta) * np.cos(phi)
    vect[1] = np.sin(theta) * np.sin(phi)
    vect[2] = np.cos(theta)
    return vect

def rand_pos(grid_size):
    """
    A random position generator.
    
    Params
    ------
    grid_size : float
        the scale of the grid of coordinates for the trap
    
    Return: np.array
        a 3x1 array filled with random positions chosen from the interval [0, 0.5 * grid_size)
    """
    return (np.random.random_sample(3) - 0.5) * grid_size

def initialize_ions(N, start_area, T):
    """
    Initializes 
    
    Params
    ------
    N : int
        the number of barium ions to initialize
    start_area : float
        the grid size of the trap
    T : float
        the temperature of the barium ions
    
    Return
    ------
    IC : np.array
        a 6xN array containing the concatenated position and velocity vectors for each ion.
        positions are indexed at IC[3*i:3*i+2] and velocities are indexed at IC[3*N+3*i:3*N+3*i+2].
        i.e. for ions 1 and 2, IC = [x_1, y_1, z_1, x_2, y_2, z_2, vx_1, vy_1, vz_1, vx_2, vy_2, vz_2]
    """
    mu_barium = np.sqrt((8*const.k*T)/(np.pi*const.mb))
    sigma_barium = np.sqrt((const.k*T)/(2*const.mb))
    # Initialize the output array
    IC = np.zeros(6*N)
    
    # Loop through the ions
    for i in range(1,N+1):
        # TODO: does this really provide the same computational efficiency as C?
        # Init random position for ion i
        IC[3*(i-1):3*(i-1)+3] = rand_pos(start_area)
        
        # Init random velocity for ion i
        IC[3*N+3*(i-1):3*N+3*(i-1)+3] = rand_3D_vect() * rand_v(mu_barium, sigma_barium)
        
    return IC

NameError: name 'const' is not defined

In [12]:
# Constant for calculating the coulomb force
kappa = const.e**2/(4*np.pi*const.epsilon_0)

@njit
def Coulomb_vect(X, N):
    """
    Takes a vector of ion positions in Nx1 format and calculates the coulomb force on each ion.
    
    Params
    ------
    X : np.array
        a vector of ion positions
    N : int
        the number of ions in the trap
    
    Return
    ------
    F : np.array
        a 3xN array of coulomb force vectors for N ions
    """
    F = np.zeros((N,3))
    X3 = X.reshape(N,3)
     
    for i in prange(1, N):
        # Calculate the permutation
        perm = X3 - np.roll(X3, 3*i)

        # Calculate the coulomb forces
        F += kappa*perm/((np.sqrt((perm**2).sum(axis=1)).repeat(3).reshape((N,3))))**3
        
    # Return flattened array
    return F.ravel()

@njit
def F_DC(X, N):
    """
    Creates a fake dc force (to be replaced by real one soon) 
    somehow setting values too high can cause simulation not to work
    
    Params
    ------
    X : np.array
        a vector of ion positions
    N : int
        the number of ions in the trap
    
    Return
    ------
    F : np.array
        a 3xN array of DC force vectors for N ions
    """
    # Reshape input array
    X3 = X.reshape((N,3))
    
    # Initialiize the output array
    F = np.zeros((N,3))
    
    # Choose amount to increase/decrease forces by
    wx = -.14e6*2*np.pi
    wy = .14e6*2*np.pi
    wz = .8e6*np.pi
    
    # Set the order of forces
    trap = np.array([wx,wy,wz])
    
    # Set the sign of forces
    order = np.array([1,-1,1])
    
    # Calculate the force
    F -= const.mb*trap**2*order*X3
    
    # Return the flattened array
    return F.ravel()

# Hyperbolic trap parameters
r0 = 0.002
z0 = 0.0005
HVolt = 800

# RF drive frequency
Omega = 12.46e6 * 2 * np.pi

@njit
def FHYP_pseudo(X,N):
    """
    which force is this?
    
    Params
    ------
    X : np.array
        a 3xN vector of ion positions
    N : int
        the number of ions in the trap
    Return
    ------
    F : np.array
        a 3xN array of force vectors for N ions
    """
    # Reshape the coordinate array to xyz coords of each ion
    Y = X.reshape((N,3))
    
    # Separate the coordinates
    x = Y[:,0]
    y = Y[:,1]
    z = Y[:,2]
    coeff = const.e**2*HVolt**2/(const.mb*Omega**2*(r0**2+2*z0**2)**2)
    
    # Convert to polar coordinates
    R = np.sqrt(x**2+y**2)
    phi = np.arctan2(y,x)
      
    FR = coeff*-2*R
    FZ = -8*coeff*z
      
    F = np.stack((FR*np.cos(phi),FR*np.sin(phi),FZ))
    
    return F.transpose(1,0).ravel()

# Laser wavelength
wav = 493e-9

# Laser wave number
k_number = 2*np.pi/wav

# Laser wave vector 
dx = .1
dy = .1
dz = .04
k_vect = 1/np.sqrt(dx**2+dy**2+dz**2)*(np.array((dx,dy,dz)))

# Big K?
K = k_vect*k_number

# Excited state lifetime for p1/2 state
tau = 8.1e-9 ####I need an actual reference for this
gamma = 1/tau

@njit
def laser_vect(V,N):
    """
    uses foote et al paper for reference, what is this?
    
    Params
    ------
    V : ???
    N : ???
    
    Return
    ------
    F : np.array
        a 3xN array of force vectors for N ions
    """
    # Initialize output array
    F = np.zeros((N,3))
    
    # What are these?
    vel = V.reshape((N,3))
    s0 = 0.25
    F0 = 0
    
    # Project velocity into laser direction
    speedk = -vel.dot(k_vect)
    delta = -200e6*2*np.pi+k_number*speedk*0
    delta = 0
    S = s0/(1+(2*delta/gamma)**2)
    S = np.zeros((N,1))
    
    # leave out F0 for now HUH???? its right there????
    F0 = const.hbar*K*gamma*S/2/(1+S)
    F += F0
    
    # Calculate recoil
    F += const.hbar*S/(1+S)*(.5*gamma*K)
      
    # Flatten array
    F += np.kron(S,k_vect) # what is np.kron?????
    F = F.ravel()
    
    # Damping coefficient
    F.ravel() # inplace double ravel ???????
    Beta = -const.hbar*4*s0*delta/gamma/(1+s0+(2*delta/gamma)**2)**2*np.kron(vel.dot(K),K)
    F -= Beta*np.kron(vel.dot(k_vect),k_vect)
    
    return F

@njit
def collisions(V):
    """
    UNKNOWN?
    
    Params
    ------
    V : ???
    
    Return
    ------
    X : ???
    """
    X = np.zeros(len(V))
    N = int(len(V)/3)
    for i in range(0,N):
        Y = V[3*i:3*i+3]
        X[3*i:3*i+3] = collision(Y)
    return X

@njit
def collision(v_old):
    """
    UNKNOWN?
    
    Params
    ------
    v_old : ???
    
    Return
    ------
    ??? : ???
    """
    vg = rand_v(mu,sigma)
    n0 = rand_3D_vect() ####random velocity unit vector assigned to ion
    n1 = rand_3D_vect() ####random velocity of gas particle
    return mg/(const.mb+mg)*np.abs(np.linalg.norm(v_old)-vg)*n0+(const.mb*v_old+mg*vg*n1)/(const.mb+mg)

In [13]:
@njit
def leap_frog(N, T, tstep, IC):
    """
    UNKNOWN?
    
    Params
    ------
    N : int
        the number of ions to simulate in the trap
    T : float
        the temperature of the trap/ions?
    tstep : float
        the integration time of the sensor
    IC : np.array
        a 6xN array of concatenated position and velocity vectors
    
    Return
    ------
    time_elapsed : np.array
        redundant?
    trajectories : np.array
        a (6, N, iterations) array for the trajectories of the ions at every iteration 
    """
    # Mass of barium?
    mb = const.mb
    
    # Determine the number of iterations
    iterations = int(T / tstep)
    time_elapsed = np.zeros(iterations)
    
    # Initialize trajectories
    trajectories = np.zeros((6*N,iterations), dtype=np.float64)
    trajectories[:,0] = IC
    pos = np.zeros(3*N)
    vel = np.zeros(3*N)

    # Initialize acceleration variable. keep track of old/new
    acc = np.zeros((3*N,2))
    t = 0
    pos = trajectories[0:3*N,0].copy()
    vel = trajectories[3*N:6*N,0]
    acc[:,1] = Coulomb_vect(pos, N)/mb + F_DC(pos,N)/mb + FHYP_pseudo(pos, N)/mb
    it = 1
    t = tstep
      
    while it < iterations:
        # Get current position, velocity of all ions
        acc[:,0] = acc[:,1].copy()
        pos = trajectories[0:3*N,it-1].copy()
        vel = trajectories[3*N:6*N,it-1].copy()
        
        # Update positions based on x=vit+1/2at^2
        trajectories[0:3*N,it] = pos+vel*tstep+.5*tstep**2*acc[:,0].copy()
        ########record old acceleration, calculate new one
          
        # Sum up all forces:
        # with micromotion
        acc[:,1] = 1/mb*(Coulomb_vect(pos,N) + F_DC(pos,N)+FHYP_pseudo(pos,N))+laser_vect(vel,N)
        
        # without micromotion
        # acc[:,1]=1/mb*(Coulomb_vect(pos,N)  +F_DC(pos,N)+FHYP_pseudo(pos,N) )

        # Compute velocities
        trajectories[3*N:6*N,it] = vel+.5*tstep*(acc[:,0].copy()+acc[:,1].copy())
        trajectories[3*N:6*N,it] = collisions(trajectories[3*N:6*N,it].copy())
        time_elapsed[it] = t
        it += 1
        t += tstep
        
    return time_elapsed, trajectories

In [14]:
# TODO: messy, clean up

# Load a file. If True you will get prompted to open one. if False, it will use random initial conditions
load_file = False

# Set file name for saving after
path = 'change_name.npy'

# Number of ions to model
N = 6

"""
Integration Parameters
"""
# Integration step
# t_int=5e-9

# set integration time to 1/20 of period
t_int = 1/(const.Omega/2/np.pi)/20

# If not loading from a file: -------
# total time
Tfinal = 0.005

# Timestep at which to record data (cant be lower than t_int and should be a multiple of it)
t_step = 2 * t_int

# Time variable to start at (so you don't record the whole cooling part if you don;t want to)
t_start = 0

#Times at which to integrate
trange = [0, Tfinal]

# Times to record data
t2 = np.arange(t_start, Tfinal, t_int)
# ------------

# Mass of cold gas for ccd reproduction images
mg = const.mb / 100
T = 2e-3

# Mean and std of boltzmann distribution for virtual gas
mu = np.sqrt((8 * const.k * T)/(np.pi * mg))
sigma = np.sqrt((const.k * T)/(2 * mg))

"""
Initial Conditions Parameters
"""
# Initial conditions of barium ions, temperature, boltzman mean and std
Tb = 150

# Size of grid where initial positions may start
start_area = 200e-6

# Random initial conditions
IC = initialize_ions(N, start_area, Tb)

start = time.time()
print("simulation has started")

# Using the leap frog algorithm (simulate trajectories with no micromotion)
time_elapsed, trajectories = leap_frog(N, Tfinal, t_int, np.array(IC))
print("time elapsed:", time_elapsed)
print("trajectories:", trajectories)

print("simulation has finished and took", time.time() - start, "s")

simulation has started


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'const': Cannot determine Numba type of <class '__main__.constants'>

File "../../../tmp/ipykernel_2740625/3688000045.py", line 25:
<source missing, REPL/exec in use?>


In [ ]:
def get_final_positions_and_velocities(trajectories, N):
    """
    Params
    ------
    trajectories : np.array
        a (6, N, iterations) array for the trajectories of the ions at every iteration
    N : int
        the number of ions in the trap
    
    Return
    ------
    fpos : tuple -> (np.array, np.array, np.array)
        the final positions of the ions, (xcoords, ycoords, zcoords)
    fvel : tuple -> (np.array, np.array, np.array)
        the final velocities of the ions, (xvels, yvels, zvels)
    """
    xcord = np.zeros(N, dtype=np.float64)
    ycord = np.zeros(N, dtype=np.float64)
    zcord = np.zeros(N, dtype=np.float64)
    
    xvel = np.zeros(N, dtype=np.float64)
    yvel = np.zeros(N, dtype=np.float64)
    zvel = np.zeros(N, dtype=np.float64)
    
    for i in range(0, N):
        # Get final position coordinates
        xcord[i] = trajectories[3*i, -1]
        ycord[i] = trajectories[3*i + 1, -1]
        zcord[i] = trajectories[3*i + 2, -1]
    
        # Get final velocity coordinates
        xvel[i] = trajectories[3*N+3*i, -1]
        yvel[i] = trajectories[3*N+3*i + 1, -1]
        zvel[i] = trajectories[3*N+3*i + 2, -1]
        
    fpos = (xcord, ycord, zcord)
    fvel = (xvel, yvel, zvel)
    return fpos, fvel

def plot_ions(fpos, fvel, pscale=30e-6):
    """
    Params
    ------
    fpos : tuple -> (np.array, np.array, np.array)
        the final positions of the ions, (xcoords, ycoords, zcoords)
    fvel : tuple -> (np.array, np.array, np.array)
        the final velocities of the ions, (xvels, yvels, zvels)
    pscale : float, optional
        the scale of the 3D plot for all axes
    """
    xcord, ycord, zcord = fpos
    xvel, yvel, zvel = fvel
    ax = plt.axes(projection='3d')
    ax.set_zlabel(r'Z', fontsize=20)
    ax.set_xlim3d(-pscale, pscale)
    ax.set_ylim3d(-pscale, pscale)
    ax.set_zlim3d(-pscale, pscale)
    ax.scatter3D(xcord, ycord, zcord)

    for i in range(0, N):
        vel_i = np.zeros(3)
        vel_i[0], vel_i[1], vel_i[2] = xvel[i], yvel[i], zvel[i]
        vel_i *= pscale
        ax.arrow3D(xcord[i], ycord[i], zcord[i], vel_i[0], vel_i[1], vel_i[2], alpha=0.5)

    plt.show()
    
fpos, fvel = get_final_positions_and_velocities(trajectories, N)
plot_ions(fpos, fvel)